In [1]:
from os import listdir, mkdir
import statistics 
import face_recognition
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import shutil
import numpy as np

import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool










import numpy as np
import pandas as pd
import os
from shutil import copyfile
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import split_folders
from PIL import ImageFile
from PIL import Image
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from os.path import isfile, join
from os import listdir
from cv2 import imread
from cv2 import imwrite
from cv2 import imshow
from cv2 import waitKey
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
from cv2 import rectangle
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot
from cv2 import cvtColor
from cv2 import COLOR_RGB2BGR
from matplotlib.pyplot import figure
import matplotlib.image as mpimg
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.display import clear_output
import sys
import ipywidgets as widgets
from IPython.display import display
import os
import random
from scipy import ndarray
from os import listdir
from os.path import isfile, join
# image processing library
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io
from IPython.display import clear_output
from skimage import img_as_uint

/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/workstation/anaconda3/env

In [2]:
#setear carpetas
DATA1 = "/home/workstation/Desktop/fv/data/pics/pic1/"
DATA2 = "/home/workstation/Desktop/fv/data/pics/pic2/"
TOP_DATA = "/home/workstation/Desktop/fv/topwanted//"
ALL_DATA = "/home/workstation/Desktop/fv/allwanted/"
LOWCOUNT = "/home/workstation/Desktop/fv/lowcount/"
FACES = '/home/workstation/Desktop/fv/faces/'
FIT_DATA ="/home/workstation/Desktop/fv/fit/"
val_dir = "/home/workstation/Desktop/fv/fit/val/"
train_dir = "/home/workstation/Desktop/fv/fit/train/"
test_dir = "/home/workstation/Desktop/fv/fit/test/"
api = "/home/workstation/Desktop/fv/topwanted/0/"
carpetas = [ DATA1, DATA2, TOP_DATA, ALL_DATA, LOWCOUNT, FACES, FIT_DATA,val_dir ,train_dir ,test_dir]

for carpeta in carpetas:
    if not os.path.exists(carpeta):
        os.mkdir(carpeta)
        
predict = '/home/workstation/Desktop/fv/perdictions/preds/'
crops = '/home/workstation/Desktop/fv/predictions/crops/'

In [ ]:

base_dir = "/home/workstation/Desktop/fv/allwanted/"
names = [k for k in listdir(base_dir)]

def todos_vs_todos(name_list):
    for name in name_list:
        files = [l for l in listdir(base_dir+name+"/")]       
        encodings = {}
        try:
            mkdir("/home/workstation/Desktop/fv/api/"+name)
        except:
            pass
        try:
            for file in files:
                picture_of_me = face_recognition.load_image_file(base_dir+name+"/"+file)
                my_face_encoding = face_recognition.face_encodings(picture_of_me)       
                if len(my_face_encoding) > 0:
                    encodings[file[:-4]] = my_face_encoding[0]
            para_el_prome = {}
            for encode in encodings:
                results = {}
                for key in encodings.keys():
                    result = face_recognition.face_distance([encodings[key]], encodings[encode])
                    results[key] = result[0]
                results = pd.DataFrame(results.items())
                results = results.sort_values(by=1, ascending=True)
                x = statistics.mean(results[1])
                if x<.50:
                    shutil.move(base_dir+name+"/"+encode+".jpg", "/home/workstation/Desktop/fv/api/"+name+"/"+encode+".jpg")
        except:
            print(file)
cores = mp.cpu_count()
df_split = np.array_split(names, cores, axis=0)
pool = Pool(cores)
df_out = np.vstack(pool.map(todos_vs_todos, df_split))
pool.close()
pool.join()
pool.clear()


#aqui tocaria revisar a mano las carpetas y ver quienes no fueron incluidos

In [ ]:
# draw each face separately
names = [l for l in listdir(TOP_DATA) if os.path.isdir(join(TOP_DATA,l)) ]

# draw each face separately
def draw_faces(filename, result_list):
    # load the image
    data = pyplot.imread(filename)
    # plot each face as a subplot
    k = 1
    for i in range(len(result_list)):
        # get coordinates
        x1, y1, width, height = result_list[i]['box']
        x2, y2 = x1 + width, y1 + height
        if x1 <0:
            x1=0
        if x2 <0:
            x2=0
        if y1 <0:
            y1=0
        if y2 <0:
            y2=0
        # define subplot
        pyplot.subplot(1, len(result_list), i+1)
        pyplot.axis('off')
        # plot face
        lalala = cvtColor(data[y1:y2, x1:x2], COLOR_RGB2BGR)
        outfile = '%s/%s_%s' % (FACES+name, k, file)
        imwrite(outfile,lalala)
        k +=1
final_val = len(names)
progress = widgets.IntProgress(value=0, min=0, max=final_val)
display(progress)
detector = MTCNN()
for name in names:
    progress.value +=1
    
    try:
        onlyfiles = [ f for f in listdir(os.path.join(TOP_DATA,name)) ]
        os.mkdir(os.path.join(FACES,name))
        for file in onlyfiles:
            filename = '%s/%s' % (TOP_DATA+name, file)
            # load image from file
            pixels = pyplot.imread(filename)
            # create the detector, using default weights
            
            #detect faces in the image
            faces = detector.detect_faces(pixels)
            faces2 = faces[:1]
            for i in faces:
                if i['confidence']>.99:
                    faces2.append(i)
            k=len(faces2)
            faces2=faces2[1:k]
            # display faces on the original image
            draw_faces(filename, faces2) 
    except:  
        pass
#nota: verificar caras dobles y errores en topwanted talves con una distancia de coceno


In [3]:
#divicion para train val y test
split_folders.ratio(FACES, output=FIT_DATA, seed=1337, ratio=(.75, .15, .1))

Copying files: 103302 files [00:18, 5653.80 files/s] 


In [4]:
import os
import random
from scipy import ndarray
from os import listdir
from os.path import isfile, join
# image processing library
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io
from IPython.display import clear_output
from skimage import img_as_ubyte


def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

# dictionary of the transformations we defined earlier
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise
}

folder_path = train_dir
names = [l for l in listdir(folder_path) if os.path.isdir(join(folder_path,l)) ]
num_files_desired = 1000

final_val = len(names)
progress = widgets.IntProgress(value=0, min=0, max=final_val)
display(progress)

for name in names:
    progress.value +=1
    
    # find all files paths from the folder
    images = [os.path.join(folder_path+name, f) for f in os.listdir(folder_path+name) if os.path.isfile(os.path.join(folder_path+name, f))]
    num_generated_files = len(images)
    while num_generated_files <= num_files_desired:
        
        
        # random image from the folder
        image_path = random.choice(images)
        # read image as an two dimensional array of pixels
        image_to_transform = sk.io.imread(image_path)
        # random num of transformation to apply
        num_transformations_to_apply = random.randint(1, len(available_transformations))

        num_transformations = 0
        transformed_image = None
        while num_transformations <= num_transformations_to_apply:
            # random transformation to apply for a single image
            key = random.choice(list(available_transformations))
            transformed_image = available_transformations[key](image_to_transform)
            num_transformations += 1
        new_file_path = '%s/augmented_image_%s.jpg' % (folder_path+name, num_generated_files)       
        # write image to the disk
        io.imsave(new_file_path, img_as_ubyte(transformed_image))
        num_generated_files += 1

IntProgress(value=0, max=1000)

/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: /home/workstation/Desktop/fv/fit/train/CRUZ RAMON ALMEIDA MACIAS/augmented_image_95.jpg is a low contrast image
/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: /home/workstation/Desktop/fv/fit/train/CRUZ RAMON ALMEIDA MACIAS/augmented_image_652.jpg is a low contrast image
/home/workstation/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: /home/workstation/Desktop/fv/fit/train/JUAN ANTONIO FERNANDEZ GARCIA/augmented_image_532.jpg is a low contrast image
